### using all packages

In [1]:
using Pkg
Pkg.DEFAULT_IO[] = stdout
Pkg.status()

     Project demo v0.1.0
      Status `~/Documents/julia/demo/Project.toml`
  [336ed68f] CSV v0.9.6
  [7876af07] Example v0.5.4 `~/.julia/dev/Example`
  [ced58d7a] HighDimMixedModels v0.1.0 `~/.julia/dev/HighDimMixedModels`
  [14b8a8f1] PkgTemplates v0.7.19
  [bd369af6] Tables v1.6.0
  [44d3d7a6] Weave v0.10.10
  [508b3551] highDimMM v0.1.0 `~/.julia/dev/highDimMM`


In [2]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels

### using module

In [3]:
using HighDimMixedModels

┌ Info: Precompiling HighDimMixedModels [ced58d7a-d58c-464d-86d9-3a8462bebd28]
└ @ Base loading.jl:1317


In [4]:
#include("HighDimMixedModels.jl")
#import .HighDimMM   #https://docs.julialang.org/en/v1/manual/variables-and-scoping/

**Set up data and formula, specify factor variables(CategoricalTerm)**

In [ ]:
#df = DataFrame(y = rand(rng, 9), a = 1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = vcat(repeat([1,2], 4),1))
df = DataFrame(y = rand(rng, 9), a = 1:1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = rand(rng, 9))
f = @formula(y ~  a + b + c + d)
contrasts = Dict( :a => ContinuousTerm, :b=>ContinuousTerm, :c => CategoricalTerm)
df

In [5]:
df = DataFrame(MixedModels.dataset(:cbpp))
first(df,5)

,herd,period,incid,hsz
,String,String,Int8,Int8
1,H01,1,2,14
2,H01,2,3,12
3,H01,3,4,9
4,H01,4,0,5
5,H02,1,3,22


In [6]:
#MixedModels.datasets()
df = DataFrame(MixedModels.dataset(:cbpp))
df[!,:period] = map(x->parse(Float64,x),df[:,:period])
first(df,5)

,herd,period,incid,hsz
,String,Float64,Int8,Int8
1,H01,1.0,2,14
2,H01,2.0,3,12
3,H01,3.0,4,9
4,H01,4.0,0,5
5,H02,1.0,3,22


In [7]:
select!(df,[:period,:herd,:incid,:hsz])
first(df,5)

,period,herd,incid,hsz
,Float64,String,Int8,Int8
1,1.0,H01,2,14
2,2.0,H01,3,12
3,3.0,H01,4,9
4,4.0,H01,0,5
5,1.0,H02,3,22


In [8]:
#f = @formula(period ~ 0 + herd + incid + hsz)
f = @formula(period ~ 0 +(1|herd) + incid + hsz )
#f = @formula(period ~ herd + incid + hsz)
#contrasts = Dict( :incid => ContinuousTerm, :hsz => ContinuousTerm, :herd => CategoricalTerm)
#fit(MixedModel, @formula(period ~ incid + hsz +(1|herd)), df, REML = true)

FormulaTerm
Response:
  period(unknown)
Predictors:
  0
  (herd)->1 | herd
  incid(unknown)
  hsz(unknown)

### Construct object

#### 1.Number of variables indicates M,X,Z

In [ ]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, 1)

#### 2.indices indicates M,X,Z

In [ ]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, [2,4],3)

In [ ]:
HMM = highDimMixedModel(f, df, contrasts, 2, 3, 1)

#### 3.variable names indicates M,X,Z

In [ ]:
#HMM = HighDimMM.highDimMixedModel(f, df, contrasts, "a", ["b","d"],"c")
HMM = highDimMixedModel(f, df, contrasts, "incid", "hsz", "herd")

In [ ]:
first(HMM.M.M,5)

In [ ]:
first(HMM.X.X,5)

In [ ]:
size(HMM.Z.Z)

In [ ]:
y, pred = modelcols(form, df);
terms = form.rhs.terms
M = highDimMat(modelmatrix(terms[idOfHDM],df))
X = XMat(modelmatrix(terms[idOfXMat],df))

#preTerms = Vector(1:length(form.rhs.terms))
#idOfReMat = preTerms[preTerms .∉ Ref(vcat(idOfHDM, idOfXmat))]
Z = ReMat(modelmatrix(terms[idOfReMat],df))

#### 4. parse random effect in formula

In [9]:
HMM = highDimMixedModel(f, df, 1)

┌ Warning: n >= p in high dimensional matrix
└ @ HighDimMixedModels /Users/zyxu/.julia/dev/HighDimMixedModels/src/bricks.jl:12


In [10]:
first(HMM.M.M,5)

5-element Vector{Float64}:
 2.0
 3.0
 4.0
 0.0
 3.0

In [11]:
first(HMM.X.X,5)

5-element Vector{Float64}:
 14.0
 12.0
  9.0
  5.0
 22.0

In [12]:
size(HMM.Z.Z)

(56, 15)

### Optimization

$$
\mathbf{y} \text { is } N_{n}(\mathbf{X} \boldsymbol{\beta}, \mathbf{\Sigma}), \quad \text { where } \quad \mathbf{\Sigma}= \sigma_{z}^{2}
\mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} 
$$

Let $$\mathbf{P}=\mathbf{I}-\mathbf{H}=\mathbf{I}-\mathbf{X}\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-} \mathbf{X}^{\prime}$$

$$\mathbf{K}=\mathbf{C}\mathbf{P} \in R^{(n-r)\times n}$$

$$
Ky \sim N_{n-r}\left(\mathbf{0}, \mathbf{K} \mathbf{\Sigma} \mathbf{K}^{\prime}\right) = N_{n-r}\left[\mathbf{0}, \mathbf{K}\left(\sigma_{z}^{2} \mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} \right) \mathbf{K}^{\prime}\right]
$$
To simplify, we assume fixed effect as full rank, where r is the number of fixed effects.

The objective function is:
$$
\frac{n-r}{2} \ln (2 \pi)-\frac{1}{2} \ln \left|\mathbf{K} \Sigma \mathbf{K}^{\prime}\right|-\frac{1}{2} \mathbf{y}^{\prime} \mathbf{K}^{\prime}\left(\mathbf{K} \Sigma \mathbf{K}^{\prime}\right)^{-1} \mathbf{K} \mathbf{y}
$$

In [ ]:
#include("HighDimMixedModels.jl")
#import .HighDimMM   #https://docs.julialang.org/en/v1/manual/variables-and-scoping/
#HMM = HighDimMM.highDimMixedModel(f, df, contrasts, "incid", "hsz","herd")

In [15]:
HMM::highDimMixedModel

In [16]:
HMM::MixedModel

In [17]:
sigma, betaM, betaX, opt = fit(HMM, verbose = true, REML = true, alg = :LN_BOBYQA) # :LN_BOBYQA :LN_COBYLA
println("")

sigma = [1.0, 1.0]
negLog = -57.1099018030012
The initial object value is -57.1099018030012
sigma = [1.0, 1.0]
negLog = -57.1099018030012
OPTBL: starting point [1.0, 1.0]
sigma = [1.0, 1.0]
negLog = -57.1099018030012
sigma = [1.75, 1.0]
negLog = -55.97912076973617
sigma = [1.0, 1.75]
negLog = -52.37720093978967
sigma = [0.25, 1.0]
negLog = -64.50473446075652
sigma = [1.0, 0.25]
negLog = -128.22481961167495
sigma = [0.29289321881345254, 0.0]
negLog = Inf
sigma = [0.9469669914110089, 0.19696699141100893]
negLog = -156.76581599673838
sigma = [0.995676667220638, 0.13993745974082217]
negLog = -207.31013755591323
sigma = [0.9952996645647639, 0.06493840728682465]
negLog = -405.77399467791014
sigma = [0.9878315127277438, 0.06424796674686986]
negLog = -410.13655848976134
sigma = [0.987341243774038, 0.05676400814509819]
negLog = -458.32682425399554
sigma = [0.9948025184365511, 0.05600283772361585]
negLog = -463.4181543729141
sigma = [0.9947292842170775, 0.05525642178789495]
negLog = -469.0572593

In [ ]:
HMM.optsum

In [ ]:
opt

In [ ]:
display(HMM)

In [ ]:
opt

In [ ]:
show(opt)

In [ ]:
display(opt)

In [ ]:
sigma

After getting estimate of sigma, we estimate beta
$$
\hat{\beta}(\theta) = (X^T\Sigma^{-1}X)^{-1}X^T\Sigma^{-1}y
$$

In [ ]:
beta

In [ ]:
data = MixedModels.dataset(:cbpp)

In [ ]:
using CSV

In [ ]:
CSV.write("cbpp",data)

In [ ]:
pwd

In [ ]:
isnothing(12)

In [ ]:
Type{<:MixedModel}

In [ ]:
using Pkg

In [ ]:
print(Pkg.status())